# *COVID-19 Interactive Analysis Dashboard (Global)*

*This notebook covers the following -*
1. *What is COVID-19?*
2. *Data loading from John Hopkins CSSE data repository*
3. *Data Cleaning and Preparation*
4. *Visualising N number of worst hit countries using plotly scatter plot.*
5. *Plotting confirmed and death cases for the requested country.*
6. *How COVID-19 spreads?*
7. *Cummulative COVID-19 World trend*
8. *Incremental COVID-19 World trend*
9. *Displaying Total Stats*
10. *Plotting all cases on world map using Folium*
<br/>

## *What is COVID-19?*

*Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as **Severe Acute Respiratory Syndrome (SARS)** and **Middle Eastern Respiratory Syndrome (MERS)**.*

*'Novel coronavirus' is a new, previously unidentified strain of coronavirus. The novel coronavirus involved in the current outbreak has been named **SARS-CoV-2** by the **World Health Organization (WHO)**.*

*The disease it causes has been named “**Coronavirus disease 2019**” (or “**COVID-19**”).*
<br/>

*Most people who fall sick with COVID-19 will experience mild to moderate symptoms and recover without special treatment. However, some will become seriously ill and require medical attention.*

<img src='./COVID-19.jpg' alt='Covid'/>
<br/>
<br/>

In [1]:
# Import packages

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.core.display import display, HTML

import seaborn as sns

import numpy as np
import pandas as pd
import folium
import math

import plotly.express as px
import plotly.graph_objects as go

import requests

## *Displaying Total Stats (Confirmed, Death and Recovered Cases)*

### *Data*
*The data used in this project is taken from **COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University**.*


In [2]:
# Loading data right from the source

death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
# Data cleaning

# Renaming the df column names to lowercase

country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

# Changing province/state to state and country/region to country

confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'Country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'Country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'Country'})
country_df = country_df.rename(columns={'country_region': 'Country'})


In [4]:
columns = ['Dates', 'Total_Infected', 'Total_deaths', 'Total_recovered']

world_df = pd.DataFrame(columns=columns)
world_df['Dates'] = confirmed_df.columns[4: 565]

for i in range(len(world_df)):
    world_df['Total_Infected'][i] = confirmed_df[world_df['Dates'][i]].sum()
    world_df['Total_deaths'][i] = death_df[world_df['Dates'][i]].sum()
    world_df['Total_recovered'][i] = recovered_df[world_df['Dates'][i]].sum()

In [5]:
confirmed_total = int(confirmed_df['10/31/21'].sum())
deaths_total = int(death_df['10/31/21'].sum())
recovered_total = list(world_df['Total_recovered'])[-1]

In [6]:
# Displaying the total stats

display(HTML("<div style = 'background-color: dimgray; padding: 20px '>" +
             "<span style='color: #fff; font-size:20px;'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: red; font-size:20px;margin-left:10px;'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: lightgreen; font-size:20px; margin-left:10px;'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )

<br/>

In [7]:
# Sorting in descending order

sorted_country_df = country_df.sort_values('confirmed', ascending=False).head(5)

## *Plotting COVID-19 Cases for the worst hit countries*

In [9]:
# Sorting in descending order

sorted_country = country_df.sort_values('confirmed', ascending=False)

In [10]:
def bubble_chart(n):
    fig = px.scatter(sorted_country.head(n), x="Country", y="confirmed", size="confirmed", color="Country",
               hover_name="Country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.bubble_chart(n)>

<br/>

## *Confirmed and Death Cases for Countries*

In [11]:
def plot_cases_for_country(Country):
    labels = ['confirmed', 'death']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if Country == 'World':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis=0)
        
        else:
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df.Country == Country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines+markers',
                                name = labels[i],
                                line = dict(color = colors[i], width = line_size[i]),
                                connectgaps = True,
                                text = 'Total ' + str(labels[i]) + ": " + str(y_data[-1])))
        
    fig.update_layout(
        title="COVID 19 cases of " + Country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="gainsboro",
        width = 800,     
    );
    
    fig.update_yaxes(type="linear")
        
    fig.show()
        

In [12]:
countries = list(country_df['Country'])
countries.insert(0,'World')


### *Check the details of your Country/ World - Choose any country from the drop-down menu*

In [13]:
interact(plot_cases_for_country, Country=countries)

interactive(children=(Dropdown(description='Country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

<function __main__.plot_cases_for_country(Country)>

<br/>

## *10 Worst Hit Countries - Confirmed Cases*

In [15]:
px.bar(
    sorted_country.head(10),
    x = "Country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## *10 Worst Hit Countries - Death Cases*

In [16]:
px.bar(
    sorted_country.head(10),
    x = "Country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["dimgray"], 
    height=500,
    width=800
)

<br/>

## *How COVID-19 spreads?*
*The virus can spread from an infected person’s mouth or nose in small liquid particles when they cough, sneeze, speak, sing or breathe. These particles range from larger respiratory droplets to smaller aerosols.*

*You can be infected by breathing in the virus if you are near someone who has COVID-19, or by touching a contaminated surface and then your eyes, nose or mouth. The virus spreads more easily indoors and in crowded settings.*
<br/>


## *Bar Charts Depicting Cummulative COVID-19 World trend*

In [17]:
# Creating new dataframe

columns = ['Dates', 'Total_Infected', 'Total_deaths', 'Total_recovered']

world_df = pd.DataFrame(columns=columns)
world_df['Dates'] = confirmed_df.columns[4: 565]

for i in range(len(world_df)):
    world_df['Total_Infected'][i] = confirmed_df[world_df['Dates'][i]].sum()
    world_df['Total_deaths'][i] = death_df[world_df['Dates'][i]].sum()
    world_df['Total_recovered'][i] = recovered_df[world_df['Dates'][i]].sum()

In [18]:
x = world_df['Dates']
y = world_df['Total_Infected']
z = world_df['Total_deaths']
w = world_df['Total_recovered']

In [19]:
# Call Bar charts and assign to fig_t. Add traces to the same figure

fig_t = go.Figure(go.Bar(x=x, y=y, name='Total Infected', marker_color='indianred', opacity=.8))

fig_t.add_trace(go.Bar(x=x, y=y-z-w, name='Total Active', marker_color='mediumblue', opacity=1))
fig_t.add_trace(go.Bar(x=x, y=w, name='Total recovered', marker_color='lightseagreen', opacity=0.8))
fig_t.add_trace(go.Bar(x=x, y=z, name='Total deaths', marker_color='black', opacity=1))

# Here we define layout of the chart

fig_t.update_layout(barmode='overlay', xaxis={'categoryorder':'total ascending'},xaxis_type='category',
                  title={
        'text': 'Cummulative COVID-19 world trend',
        'y':0.79,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'},)
fig_t.update_xaxes(title= 'Timeline' ,showline=True)
fig_t.update_yaxes(title= 'Number of cases', showline=True)
fig_t.show()

## *Line Charts Depicting Incremental COVID-19 World trend*

In [20]:
# Creating new dataframe

df = world_df

df['Total_Infected'] = world_df['Total_Infected'].diff()
df['Total_deaths'] = world_df['Total_deaths'].diff()
df['Total_recovered'] = world_df['Total_recovered'].diff()

df = df.iloc[1:]

In [21]:
def plot_in_range(Range):
    global df;
    
    # Call Line charts and assign to fig_inc. Add traces to the same figure
    
    if Range == '1/23/2020 - 6/10/2020':
        df_diff = df.iloc[:140]
    elif Range == '6/10/2020 - 10/28/2020':
        df_diff = df.iloc[140:280]
    elif Range == '10/28/2020 - 3/17/2021':
        df_diff = df.iloc[280:420]
    else:
        df_diff = df.iloc[420:]
    
    y = df_diff['Total_Infected']
    z = df_diff['Total_deaths']
    w = df_diff['Total_recovered']


    fig_inc = go.Figure(go.Line(x=df_diff['Dates'], y=df_diff['Total_Infected'],
                                name='Infected', mode='lines+markers',marker=dict(size=10,color='indianred')))

    fig_inc.add_trace(go.Line(x=df_diff['Dates'], y=df_diff['Total_recovered'],
                              name='Recovered', mode='lines+markers',marker=dict(size=10,color='lightseagreen')))
    fig_inc.add_trace(go.Line(x=df_diff['Dates'], y=df_diff['Total_deaths'], 
                              name='Deaths', mode='lines+markers',marker=dict(size=10,color='gray')))
    fig_inc.add_trace(go.Line(x=df_diff['Dates'], y=y-z-w, 
                              name='Active', mode='lines+markers',marker=dict(size=10,color='Orange')))

    # Here we define layout of the chart

    fig_inc.update_layout(xaxis_showgrid=True, yaxis_showgrid=True, plot_bgcolor='whitesmoke', 
            title={
            'text': 'Incremental COVID-19 world trend',
            'y':0.75,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},xaxis_type='category')
    fig_inc.update_xaxes(title= 'Timeline' ,showline=False)
    fig_inc.update_yaxes(title= 'Number of incremental cases', showline=False)

    fig_inc.show()

### *Choose a range from the drop-down menu to see the Incremental Trend in COVID-19*

In [22]:
interact(plot_in_range, Range=['1/23/2020 - 6/10/2020', '6/10/2020 - 10/28/2020', 
                               '10/28/2020 - 3/17/2021', '3/17/2021 - 8/4/2021']);

interactive(children=(Dropdown(description='Range', options=('1/23/2020 - 6/10/2020', '6/10/2020 - 10/28/2020'…

<br/>

## *Global Spread of COVID-19*

In [23]:
world_map = folium.Map(location=[25,0], tiles="cartodbpositron", zoom_start = 2, max_zoom = 5, min_zoom = 2, max_bounds=True)


for i in range(0, len(confirmed_df)):
    if not math.isnan(float(confirmed_df.iloc[i]['lat'])):
        folium.Circle(fill=True, location=(confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']),
                      width='100%', height='100%', left='0%', top='0%', position='relative',
                      radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*30000,
                      color="#FF4500",
                      fill_color='indigo',
                      tooltip = "<div style='margin: 0; background-color: dimgray; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['Country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
                     ).add_to(world_map)

world_map


<br/>

<br/>

# *COVID-19 US Interactive Analysis Dashboard*

<br/>

In [27]:
# Loading data right from the source:
death_df1 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
confirmed_df1 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [28]:
confirmed_df1.set_index(confirmed_df1['Province_State'],inplace=True)
confirmed_df1.rename(columns={'Province_State': 'State'},inplace=True)
death_df1.set_index(death_df1['Province_State'],inplace=True)
death_df1.rename(columns={'Province_State': 'State'},inplace=True)

In [29]:
confirmed_df_states=confirmed_df1.drop(columns=['UID', 'code3', 'FIPS', 'Lat', 'Long_']).groupby(['State']).sum().copy()
death_df_states=death_df1.drop(columns=['UID', 'code3', 'FIPS', 'Lat', 'Long_']).groupby(['State']).sum().copy()

In [30]:
confirmed_df_states=pd.merge(confirmed_df1[['Lat', 'Long_','State']].groupby(['State']).mean(),confirmed_df_states, left_index=True, right_index=True)
death_df_states=pd.merge(death_df1[['Lat', 'Long_','State']].groupby(['State']).mean(),death_df_states, left_index=True, right_index=True)

In [31]:
combined_df=pd.DataFrame()
combined_df['Confirmed']=confirmed_df_states[confirmed_df_states.columns[-1]]
combined_df['Deaths']=death_df_states[death_df_states.columns[-1]]

## *State-wise Data*

In [32]:
sorted_combined_df=combined_df.loc[:,["Confirmed","Deaths"]].sort_values("Confirmed",ascending=False)
sorted_combined_df['State']=sorted_combined_df.index
combined_df.loc[:,["Confirmed","Deaths"]].sort_values("Confirmed",ascending=False).style.background_gradient(cmap='Blues',subset=["Confirmed"]).background_gradient(cmap='Reds',subset=["Deaths"])

,Confirmed,Deaths
State,,
California,4932199,72111
Texas,4242156,71262
Florida,3690812,59670
New York,2564789,56510
Illinois,1701649,28267
Georgia,1636325,29174
Pennsylvania,1564939,31455
Ohio,1547788,24527
North Carolina,1482464,18104


<br/>

## *10 Worst Hit States - Confirmed Cases*

In [33]:
px.bar(
    sorted_combined_df.head(10),
    x = "State",
    y = "Confirmed",
    title= "Top 10 worst affected States", # the axis names
    color_discrete_sequence=["Orange"], 
    height=500,
    width=800
)

## *10 Worst Hit States - Death Cases*

In [34]:
px.bar(
    sorted_combined_df.head(10),
    x = "State",
    y = "Deaths",
    title= "Top 10 worst affected States", # the axis names
    color_discrete_sequence=["red"], 
    height=500,
    width=800
)

<br/>

## *Map Visualization According to States - Confirmed Cases*

In [35]:
state_geo = requests.get('https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json').json()

In [36]:
data_temp = confirmed_df1.groupby(["State"]).sum().reset_index().drop(["Lat","Long_"],axis=1)
data_temp["Confirmed_log"] = np.log10(data_temp.iloc[:,-1]+1)
bins = list(data_temp['Confirmed_log'].quantile([0, 0.25, 0.5, 0.75,0.95 ,1]))
m = folium.Map(location=[37, -102], zoom_start=4,max_zoom=6,min_zoom=3)
 

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data = data_temp,
    columns=['State', 'Confirmed_log'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = bins,
    reset=True,
    legend_name='Confirmed cases log (10^x)'
).add_to(m)
folium.LayerControl().add_to(m)

legend_html = "<div style='padding:10px;background-color:rgba(255,255,255,0.5);position:fixed;bottom:20px;left:20px;z-index:1000'>"
legend_html += "<div style='width:100%;text-align:center;'><h4>Index for Legend</h4></div><hr style='border-top-color: rgba(25,25,25,0.5);'>"
legend_html += "<ul style='margin:0;padding:0;color: #555;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"
for i in bins:
    legend_html += "<li style='margin:0;padding:0;line-height: 0;'>Value "+str(np.round(i,2))+" : "+str(int(10**i)-1)+"</li><br>"
legend_html += "</ul></div>"
m.get_root().html.add_child(folium.Element(legend_html))
m

<br/>

## *Map Visualization According to States - Death Cases*

In [37]:
data_temp = death_df1.groupby(["State"]).sum().reset_index().drop(["Lat","Long_"],axis=1)
data_temp["Death_log"] = np.log10(data_temp.iloc[:,-1]+1)
bins = list(data_temp['Death_log'].quantile([0, 0.25, 0.5, 0.75,0.95 ,1]))
m = folium.Map(location=[37, -102], zoom_start=4,max_zoom=6,min_zoom=3)
 

folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data = data_temp,
    columns=['State', 'Death_log'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = bins,
    reset=True,
    legend_name='Death cases log (10^x)'
).add_to(m)
folium.LayerControl().add_to(m)

legend_html = "<div style='padding:10px;background-color:rgba(255,255,255,0.5);position:fixed;bottom:20px;left:20px;z-index:1000'>"
legend_html += "<div style='width:100%;text-align:center;'><h4>Index for Legend</h4></div><hr style='border-top-color: rgba(25,25,25,0.5);'>"
legend_html += "<ul style='margin:0;padding:0;color: #555;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"
for i in bins:
    legend_html += "<li style='margin:0;padding:0;line-height: 0;'>Value "+str(np.round(i,2))+" : "+str(int(10**i)-1)+"</li><br>"
legend_html += "</ul></div>"
m.get_root().html.add_child(folium.Element(legend_html))
m

<br/>

## *More Info on COVID-19*

* *https://www.who.int/health-topics/coronavirus*
* *https://www.who.int/emergencies/diseases/novel-coronavirus-2019*
* *https://www.nature.com/articles/s41597-020-0448-0*